In [2]:
import pandas as pd
import numpy as np
import PyPDF2
import string
import re
import os
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
from nltk import word_tokenize
pd.set_option('display.max_columns', 100)

In [3]:
with open('dictionary.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [19]:
Cases = pd.read_csv('data/Cases2018.csv', encoding='iso-8859-1')

In [20]:
Labels = Cases[['docket','partyWinning']]
Text = pd.DataFrame.from_dict(b, orient="index")

In [21]:
Text = Text.rename(index=str, columns={0: "Document"})

In [22]:
Text['docket'] = Text.index
Text.reset_index(drop=True)

,Document,docket
0,1 IN THE SU...,00-1011
1,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1021
2,1 2 \n ...,00-1045
3,12345678910\n111213141516171819202122232425 IN...,00-10666
4,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1072
5,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1073
6,1234\n5\n678\n910\n11\n12\n131415\n161718\n19\...,00-1089
7,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1167
8,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1187
9,1 IN THE SU...,00-121


In [23]:
Text = pd.merge(Text, Labels, on='docket', how='inner')
Text.drop(Text[Text.partyWinning==2.0].index, inplace = True)
Labels = Text[['partyWinning']]
Text.drop('partyWinning', axis=1, inplace= True)

In [24]:
Text.Document[1]

'12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\n16\n17\n18\n19\n20\n21\n22\n23\n24\n25 IN THE SUPREME COURT OF THE UNITED STATES•-- - - - - - - - - - - - - - -X•RUSH PRUDENTIAL HMO, INC., :•Petitioner :•v. : No. 00-1021•DEBORAH C. MORAN :•-- - - - - - - - - - - - - - -X•Washington, D.C.•\nWednesday, January 16, 2002•The above-entitled matter came on for oral•argument before the Supreme Court of the United States at•\n10:07 a.m.•\nAPPEARANCES:•\nJOHN G. ROBERTS, JR., ESQ., Washington, D.C.; on behalf of•the Petitioner.•DANIEL P.ALBERS, ESQ., on behalf of the Respondent.•\nEDWIN S. KNEEDLER, ESQ., Deputy Solicitor General,•Department of Justice, Washington, D.C.; on behalf of•\nthe United States, as amicus curiae.•1 Alderson Reporting Company 1111 14th Street, N.W. Suite 400 1-800-FOR-DEPO Washington, DC 20005  12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\n16\n17\n18\n19\n20\n21\n22\n23\n24\n25 C O N T E N T S•ORAL ARGUMENT OF •\nJOHN G. ROBERTS, ESQ.•On behalf of the Petitioner •ORAL A

In [25]:
Text['Document'] = Text['Document'].apply(lambda x: " ".join(x.lower() for x in x.split()))
Text['Document'].head()

0    1 in the supreme court of the united states 2 ...
1    12 3 4 5 6 7 8 910 11 12 13 14 15 16 17 18 19 ...
2    1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
3    12345678910 111213141516171819202122232425 in ...
4    12 3 4 5 6 7 8 910 11 12 13 14 15 16 17 18 19 ...
Name: Document, dtype: object

In [26]:
Text.Document[1]

'12 3 4 5 6 7 8 910 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 in the supreme court of the united states•-- - - - - - - - - - - - - - -x•rush prudential hmo, inc., :•petitioner :•v. : no. 00-1021•deborah c. moran :•-- - - - - - - - - - - - - - -x•washington, d.c.• wednesday, january 16, 2002•the above-entitled matter came on for oral•argument before the supreme court of the united states at• 10:07 a.m.• appearances:• john g. roberts, jr., esq., washington, d.c.; on behalf of•the petitioner.•daniel p.albers, esq., on behalf of the respondent.• edwin s. kneedler, esq., deputy solicitor general,•department of justice, washington, d.c.; on behalf of• the united states, as amicus curiae.•1 alderson reporting company 1111 14th street, n.w. suite 400 1-800-for-depo washington, dc 20005 12 3 4 5 6 7 8 910 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 c o n t e n t s•oral argument of • john g. roberts, esq.•on behalf of the petitioner •oral argument of• daniel p.albers, esq.•on behalf of the r

In [27]:
Text['Document'] = Text['Document'].str.replace('[^\w\s]',' ')
Text['Document'][1]

'12 3 4 5 6 7 8 910 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 in the supreme court of the united states                               x rush prudential hmo  inc     petitioner   v    no  00 1021 deborah c  moran                                 x washington  d c   wednesday  january 16  2002 the above entitled matter came on for oral argument before the supreme court of the united states at  10 07 a m   appearances   john g  roberts  jr   esq   washington  d c   on behalf of the petitioner  daniel p albers  esq   on behalf of the respondent   edwin s  kneedler  esq   deputy solicitor general  department of justice  washington  d c   on behalf of  the united states  as amicus curiae  1 alderson reporting company 1111 14th street  n w  suite 400 1 800 for depo washington  dc 20005 12 3 4 5 6 7 8 910 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 c o n t e n t s oral argument of   john g  roberts  esq  on behalf of the petitioner  oral argument of  daniel p albers  esq  on behalf of the r

In [28]:
user_defined_stop_words = ['-x','--','x•','n','w','e','washington','x','v','alderson','petitioner','respondent','reporting','company','supreme','court','united','states','states•','•v','•petitioner'] 

i = stopwords.words('english')
j = list(string.punctuation) + user_defined_stop_words

stoplist = set(i).union(j)

In [35]:
Text['Document'] = Text['Document'].apply(lambda x: " ".join(x for x in x.split() if x not in stoplist and not x.isdigit()))
Text['Document'].head()

0    deboris calcano martinez et al petitioners imm...
1    rush prudential hmo inc deborah moran wednesda...
2    25in trw inc adelaide andrews tuesday october ...
3    william joseph harris monday march entitled ma...
4    leonard edelman lynchburg college tuesday janu...
Name: Document, dtype: object

In [33]:
Text['Document'][2]

'25in trw inc 00 1045 adelaide andrews tuesday october 2001 entitled matter came oral 02 appearances glen nager esq behalf andrew r henderson esq los angeles california behalf kent l jones esq assistant solicitor general department behalf amicus curiae support inc fourteenth 202 289 2260 25c oral page glen nager esq behalf andrew r henderson esq behalf kent l jones esq behalf amicus curiae support inc fourteenth 202 289 2260 p r g 02 rehnquist hear next 00 1045 trw inc adelaide andrews nager oral glen nager behalf nager thank may please appeals ninth circuit held limitations claim improper disclosure fair credit act commence plaintiff discovers injury opposed date improper disclosure appeals ninth circuit parse language limitations fair credit act rather language sufficiently expressed overcome presumption favor discovery rule ninth circuit held read statutes injury occur nager injury alleged inc fourteenth 202 289 2260 improper disclosure occurred date alleged improper disclosure conn

In [31]:
freq = pd.Series(' '.join(Text['Document']).split()).value_counts()[:100]

In [32]:
freq = list(freq.index)
Text['Document'] = Text['Document'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
Text['Document'].head()

0    deboris calcano martinez et al petitioners 00 ...
1    rush prudential hmo inc 00 1021 deborah moran ...
2    25in trw inc 00 1045 adelaide andrews tuesday ...
3    12345678910 111213141516171819202122232425 wil...
4    leonard edelman 00 1072 lynchburg college tues...
Name: Document, dtype: object

In [38]:
freq = pd.Series(' '.join(Text['Document']).split()).value_counts()[-100:]



In [39]:
freq = list(freq.index)
Text['Document'] = Text['Document'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
Text['Document'].head()

0    deboris calcano martinez et al petitioners imm...
1    rush prudential hmo inc deborah moran wednesda...
2    25in trw inc adelaide andrews tuesday october ...
3    william joseph harris monday march entitled ma...
4    leonard edelman lynchburg college tuesday janu...
Name: Document, dtype: object

In [40]:
Text['Document'][1]

'rush prudential hmo inc deborah moran wednesday january entitled matter came oral appearances john g roberts jr esq behalf daniel p albers esq behalf edwin kneedler esq deputy solicitor general department behalf amicus curiae oral john g roberts esq behalf oral daniel p albers esq behalf oral edwin kneedler esq behalf amicus curiae rebuttal john g roberts jr esq behalf page p r g rehnquist hear first morning number rush prudential hmo inc deborah moran roberts oral john g roberts jr behalf roberts thank may please erisa preemption cases exceedingly complicated submission morning straightforward held pilot life erisa civil enforcement provisions exclusive remedy improper processing claim benefits erisa regulated plan illinois independent external issue affords different remedy beneficiary dissatisfied hmo denial benefits illinois therefore preempted last week great west life began analysis noting especially reluctant tamper erisa enforcement scheme quoting prior precedent proposition e

In [41]:
Text.Document.head()

0    deboris calcano martinez et al petitioners imm...
1    rush prudential hmo inc deborah moran wednesda...
2    25in trw inc adelaide andrews tuesday october ...
3    william joseph harris monday march entitled ma...
4    leonard edelman lynchburg college tuesday janu...
Name: Document, dtype: object

In [44]:
import string

from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag

from sklearn.base import BaseEstimator, TransformerMixin


class NLTKPreprocessor(BaseEstimator, TransformerMixin):

    def __init__(self, stopwords=None, punct=None,
                 lower=True, strip=True):
        self.lower      = lower
        self.strip      = strip
        self.stopwords  = stopwords or set(sw.words('english'))
        self.punct      = punct or set(string.punctuation)
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y=None):
        return self

    def inverse_transform(self, X):
        return [" ".join(doc) for doc in X]

    def transform(self, X):
        return [
            list(self.tokenize(doc)) for doc in X
        ]

    def tokenize(self, document):
        # Break the document into sentences
        for sent in sent_tokenize(document):
            # Break the sentence into part of speech tagged tokens
            for token, tag in pos_tag(wordpunct_tokenize(sent)):
                # Apply preprocessing to the token
                token = token.lower() if self.lower else token
                token = token.strip() if self.strip else token
                token = token.strip('_') if self.strip else token
                token = token.strip('*') if self.strip else token
                token = token.strip('•') if self.strip else token

                # If stopword, ignore token and continue
                if token in self.stopwords:
                    continue

                # If punctuation, ignore token and continue
                if all(char in self.punct for char in token):
                    continue

                # Lemmatize the token and yield
                lemma = self.lemmatize(token, tag)
                yield lemma

    def lemmatize(self, token, tag):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)

In [58]:
def build_and_evaluate(X, y, classifier=SGDClassifier):

   
    def build(classifier, X, y=None):
        """
        Inner build function that builds a single model.
        """
        if isinstance(classifier, type):
            classifier = classifier()

        model = Pipeline([
            ('preprocessor', NLTKPreprocessor()),
            ('vectorizer', TfidfVectorizer()),
            ('classifier', classifier)])

        model.fit(X, y)
        return model
    return model

In [59]:
X = Text.Document
y= Labels

In [60]:
model =build_and_evaluate(X,y, SGDClassifier)

In [63]:
Text.Document[1]

'rush prudential hmo inc deborah moran wednesday january entitled matter came oral appearances john g roberts jr esq behalf daniel p albers esq behalf edwin kneedler esq deputy solicitor general department behalf amicus curiae oral john g roberts esq behalf oral daniel p albers esq behalf oral edwin kneedler esq behalf amicus curiae rebuttal john g roberts jr esq behalf page p r g rehnquist hear first morning number rush prudential hmo inc deborah moran roberts oral john g roberts jr behalf roberts thank may please erisa preemption cases exceedingly complicated submission morning straightforward held pilot life erisa civil enforcement provisions exclusive remedy improper processing claim benefits erisa regulated plan illinois independent external issue affords different remedy beneficiary dissatisfied hmo denial benefits illinois therefore preempted last week great west life began analysis noting especially reluctant tamper erisa enforcement scheme quoting prior precedent proposition e

In [65]:
from textblob import Word


In [66]:
Text['Document'] = Text['Document'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
Text['Document'].head()

0    deboris calcano martinez et al petitioner immi...
1    rush prudential hmo inc deborah moran wednesda...
2    25in trw inc adelaide andrew tuesday october e...
3    william joseph harris monday march entitled ma...
4    leonard edelman lynchburg college tuesday janu...
Name: Document, dtype: object

'n'

'rush prudential hmo inc deborah moran wednesday january entitled matter came oral appearance john g robert jr esq behalf daniel p albers esq behalf edwin kneedler esq deputy solicitor general department behalf amicus curia oral john g robert esq behalf oral daniel p albers esq behalf oral edwin kneedler esq behalf amicus curia rebuttal john g robert jr esq behalf page p r g rehnquist hear first morning number rush prudential hmo inc deborah moran robert oral john g robert jr behalf robert thank may please erisa preemption case exceedingly complicated submission morning straightforward held pilot life erisa civil enforcement provision exclusive remedy improper processing claim benefit erisa regulated plan illinois independent external issue affords different remedy beneficiary dissatisfied hmo denial benefit illinois therefore preempted last week great west life began analysis noting especially reluctant tamper erisa enforcement scheme quoting prior precedent proposition enforcement sc

In [73]:
clf = Pipeline([('vectorizer', TfidfVectorizer()),
            ('classifier', SGDClassifier())])

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [75]:
X_train, X_test, y_train, y_test = train_test_split(Text.Document, Labels, test_size=0.2, random_state=23)
y_test = np.ravel(y_test)
y_train = np.ravel(y_train)


In [78]:
mod2 =  clf.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [80]:
preds = mod2.predict(X_test)

In [81]:
score2 = metrics.accuracy_score(y_test, preds)

In [82]:
score2

0.61003861003861

In [83]:
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

In [85]:
X_test

86      pierce county ignacio guillen legal guardian j...
952     ˆˇ ˇˆ ˇˇ ˇˆ ˇˆ ˇˇ ˆˆ ˇˇ ˇˆ ˇˇˆ ˇˇ ˇˇ ˇˆ ˇˇ ˇˇ ...
892     ˇˇ ˇˇ ˆˇ ˆˆ ˆˇ ˆˆ ˇˇ ˆˇ ˆˆ ˇˇ ˆˆ ˇˇ ˇˇ ˇˆ ˆˆ ˇ...
546     official reviewalderson company11 3jack davis ...
733     25official kevin kasten saint gobain performan...
107     jeanne woodford warden robert frederick garcea...
652     12345678910111213141516171819202122232425offic...
836     25official standard fire insurance greg knowle...
212     green tree financial corp nka conseco finance ...
659     12345678910111213141516171819202122232425offic...
587     12345678910111213141516171819202122232425offic...
1054    ˇˆ a1 2d 9f 9f 9f 9f e2 g f2 f2 f2 7g 8d 9f e2...
679     25official vincent cullen acting warden scott ...
351     states2 jose ernesto medellin doug dretke dire...
989     ˇˆ ˇˆ ˆˇ ˇˇ ˆˆ ˇˇ ˇˇ ˇˇ ˇˇ ˆˇ ˇˇ ˇˇ ˇˇ ˆˆ ˇˇ ˇ...
999     ˆˇ ˆˇ ˆˆ ˆˇ ˆˆ ˆˆ ˆˆ ˆˇ ˆˆ ˆˇ ˆˇ ˆˇ ˇˆ ˆˆ ˆˆ ˆ...
455     12345678910111213141516171819202122232425offic...
756     25offi

In [89]:
Text.Document[453]

'12345678910111213141516171819202122232425official wyeth diana levine monday november entitled matter came oral appearance seth p waxman esq behalf edwin kneedler esq deputy solicitor general department behalf amicus curia supporting david frederick esq behalf 1alderson 12345678910111213141516171819202122232425official soral page seth p waxman esq behalf 3edwin kneedler esq behalf amicus curia supporting 14david frederick esq behalf 24rebuttal ofseth p waxman esq behalf 512alderson 12345678910111213141516171819202122232425official p r g robert hear first morning wyeth levine waxman oral seth p waxman behalf waxman may please concern conflict pre emption supremacy clause conflict presented stark repeatedly year fda approved phenergan injection safe effective condition method use described labeling including referred iv push injection yet jury evaluating risk fda considered determined precise labeling fda required wyeth use fact rendered phenergan unreasonably dangerous kennedy outset ma

In [92]:
len(Text.iloc[1022])

2